In [2]:
# clean data
import re

def removeAllEmptyLine(data):
    """
    " dùng sau khi thêm nhãn vì thêm nhãn dựa vào dòng trống
    " dùng khi gộp file đã xử lý để ghi ra file
    """
    return re.sub(r'\n+', r'\n', data)

def removeHeaderLine(data):
    """
    " remove line start with number
    """
    return re.sub(r'^\d.*\n', '', data, flags=re.MULTILINE)

# def removeTitleLine(data):
#     return re.sub(r'^title:.*$\n', '', data, flags=re.IGNORECASE | re.MULTILINE)

# def removeAltDesLine(data):
#     """
#     " remove line start with (alt/des) which is the description of image
#     """
#     return re.sub(r'(?:^alt.*\n)|(?:^des.*\n)', '', data, flags=re.MULTILINE)

def clean(data):
    data = removeHeaderLine(data)
    return data

In [8]:
import os
path_train = '/workspace/nlplab/kienvt/COQE/data/VLSP23-ComOM/raw/Version 2/VLSP2023_ComOM_private_test_nolabel/'
file_names = [os.path.join(path_train, f) for f in os.listdir(path_train) if f.endswith('.txt')]
print(file_names)
data_train = []

for file in sorted(file_names):
    with open(file, 'r') as f:
        data = f.read()
        data = clean(data)
        # Thay thế phần từ đầu đến \t bằng phần sau \t đến hết câu (nếu có \t)
        data = re.sub(r'^(.*?)\t(.*)$', r'\2', data, flags=re.MULTILINE)

        data_train.append(data)
    f.close()

['/workspace/nlplab/kienvt/COQE/data/VLSP23-ComOM/raw/Version 2/VLSP2023_ComOM_private_test_nolabel/test_0013.txt', '/workspace/nlplab/kienvt/COQE/data/VLSP23-ComOM/raw/Version 2/VLSP2023_ComOM_private_test_nolabel/test_0023.txt', '/workspace/nlplab/kienvt/COQE/data/VLSP23-ComOM/raw/Version 2/VLSP2023_ComOM_private_test_nolabel/test_0005.txt', '/workspace/nlplab/kienvt/COQE/data/VLSP23-ComOM/raw/Version 2/VLSP2023_ComOM_private_test_nolabel/test_0035.txt', '/workspace/nlplab/kienvt/COQE/data/VLSP23-ComOM/raw/Version 2/VLSP2023_ComOM_private_test_nolabel/test_0024.txt', '/workspace/nlplab/kienvt/COQE/data/VLSP23-ComOM/raw/Version 2/VLSP2023_ComOM_private_test_nolabel/test_0034.txt', '/workspace/nlplab/kienvt/COQE/data/VLSP23-ComOM/raw/Version 2/VLSP2023_ComOM_private_test_nolabel/test_0030.txt', '/workspace/nlplab/kienvt/COQE/data/VLSP23-ComOM/raw/Version 2/VLSP2023_ComOM_private_test_nolabel/test_0033.txt', '/workspace/nlplab/kienvt/COQE/data/VLSP23-ComOM/raw/Version 2/VLSP2023_ComOM_p

"Và Snapdragon 450 không nhanh và mạnh như những điện thoại tầm trung hay cao cấp ." 

In [9]:
print(len(data_train))

36


In [10]:
for data in data_train:
    while data.find('  ') != -1:
        data = data.replace('  ', ' ')

In [11]:
data_train

['Title : Góc thắc mắc : Nên mua iPhone 14 hay Samsung Galaxy S22 ?\n\nHiện nay , iPhone 14 và Samsung Galaxy S22 là hai cái tên rất “ hot ” , liên tục khuấy đảo thị trường điện thoại phân khúc cao cấp .\n\nĐứng trước sự lựa chọn giữa hai “ siêu phẩm ” đến từ hai thương hiệu đình đám Apple và Samsung , nhiều người vẫn rất phân vân không biết nên lựa chọn sản phẩm nào thì tốt hơn .\n\nĐể đưa ra quyết định chọn mua đúng đắn , mời bạn tham khảo bài viết của Nguyễn Kim nhé !\n\nTháng 9 vừa qua , các tín đồ yêu công nghệ tỏ ra vô cùng thích thú vì sau thời gian dài chờ đợi thì Apple cũng đã chính thức trình làng các sản phẩm trong iPhone 14 Series với những cải tiến ấn tượng về thiết kế lẫn hiệu năng .\n\nTrong cùng phân khúc giá , đối thủ xứng tầm với iPhone 14 khiến nhiều người phải đau đầu khi chọn lựa đó chính là điện thoại Samsung Galaxy S22 .\n\nMỗi một sản phẩm điều được thương hiệu đặt hết tâm huyết vào nên sẽ có những ưu điểm nổi trội riêng .\n\nCùng Nguyễn Kim đi sâu vào các khía 

In [7]:
import random

random.shuffle(data_train)

In [9]:
train_rate, dev_rate, test_rate = 0.8, 0.2, 0.0
num_train = int(len(data_train) * train_rate)
num_test = int(len(data_train) * test_rate)
num_dev = len(data_train) - num_train - num_test
train_set, dev_set, test_set = data_train[:num_train], data_train[num_train:num_train+num_dev], data_train[num_train+num_dev:]

In [10]:
print(len(train_set), len(dev_set), len(test_set))

67 17 0


In [11]:
# add label

def removeMultipleToOnlyOneEmptyLine(data):
    return re.sub(r'\n\s*\n', r'\n\n', data, flags=re.MULTILINE)

def add_label(data):
    """
    " add label for uncomparative sentence:
    " + 1 space + 1 tab (\t) at the end + label (0: uncomparative /1: comparative)
    " + label \n[[];[];[];[];[]] (if uncomparative)
    """

    data = removeMultipleToOnlyOneEmptyLine(data)

    # add label [[];[];[];[];[]] in each empty line
    data = re.sub(r"\n\n", r"\n[[];[];[];[];[]]\n", data, flags=re.MULTILINE)

    # remove [[];[];[];[];[]] in line which already has label
    data = re.sub(r"}\n\[\[];\[];\[];\[];\[]]","}", data, flags=re.MULTILINE)

    # # add label 0 after uncomparative sentence
    data = re.sub(r"\n\[", r" \t0\n[", data, flags=re.MULTILINE)

    # # add label 1 after comparative sentence
    data = re.sub(r"([^{|\n].*)\n{", r"\1 \t1\n{", data, flags=re.MULTILINE)
    data = re.sub(r"}\s\t1", "}", data, flags=re.MULTILINE)
    # data = re.sub(r"([^{}\n]*)\n{", r"\1 \t1\n{", data, flags=re.MULTILINE)
    # data = re.sub(r"(\n[^{|\n].*)\n{", r"\1 \t1\n{", data, flags=re.MULTILINE)

    return data

In [12]:
# config label type from {} to []

import json

def replace_j_to_l(data):
    """
    " thêm comment @Mạnh
    """
    # print(data)
    result = []
    for key, value in data.items():
        # print(key, type(value))
        if isinstance(value, list):
            result.append("[" + " ".join(value) + "]")
        else:
            result.append(f"[{value}]")
    return "[" + ";".join(result).replace("\'", '') + "]"

def replace_json(match):
    """
    " thêm comment @Mạnh
    " output: label dạng []
    """
    json_str = match.group(0)
    json_obj = json.loads(json_str)

    return replace_j_to_l(json_obj)

def replaceLabelType(data):
    return re.sub(r'{.+?}', replace_json, data)

In [13]:
subset = train_set[1]
print(subset)

title : Đánh giá Samsung Galaxy M51 : Pin rất khủng , màn hình đẹp , giá rẻ

Đầu tháng 9 vừa qua , Samsung lại ra mắt thêm một sản phẩm tuyệt vời ở phân khúc tầm trung .

Đó chính là chiếc Samsung Galaxy M51 .

Mình phải nói là Samsung thực sự “ sởi lởi ” khi trang bị cho chiếc điện thoại mới này rất nhiều thứ .

Đó là một viên pin khủng tới 7000 mAh , một màn hình Super AMOLED 6.7 inch nằm ở tầm trung .

Vậy ngoài ra chiếc điện thoại này còn gì nổi bật nữa không ?

Nó có thực sự nổi bật trong phân khúc không ?

Cùng tìm hiểu chi tiết hơn về chiếc điện thoại này qua bài đánh giá dưới đây của mình .

Thông số kỹ thuật

Đánh giá chi tiết điện thoại Samsung Galaxy M51

Thiết kế

Nói về thiết kế , mình phải nói đó là một bước đột phá mạnh mẽ của Samsung .

Từ trước đến nay , chúng ta đã thấy những chiếc điện thoại pin 5000 mAh đã gặp rắc rối thiết kế vì có một thân máy dày .

Đó là một điều khó chịu khi chúng ta dùng smartphone .

alt : Samsung Galaxy M51

des : Samsung Galaxy M51

Nhưng đ

In [14]:
print(replaceLabelType(add_label(subset)))

title : Đánh giá Samsung Galaxy M51 : Pin rất khủng , màn hình đẹp , giá rẻ 	0
[[];[];[];[];[]]
Đầu tháng 9 vừa qua , Samsung lại ra mắt thêm một sản phẩm tuyệt vời ở phân khúc tầm trung . 	0
[[];[];[];[];[]]
Đó chính là chiếc Samsung Galaxy M51 . 	0
[[];[];[];[];[]]
Mình phải nói là Samsung thực sự “ sởi lởi ” khi trang bị cho chiếc điện thoại mới này rất nhiều thứ . 	0
[[];[];[];[];[]]
Đó là một viên pin khủng tới 7000 mAh , một màn hình Super AMOLED 6.7 inch nằm ở tầm trung . 	0
[[];[];[];[];[]]
Vậy ngoài ra chiếc điện thoại này còn gì nổi bật nữa không ? 	0
[[];[];[];[];[]]
Nó có thực sự nổi bật trong phân khúc không ? 	0
[[];[];[];[];[]]
Cùng tìm hiểu chi tiết hơn về chiếc điện thoại này qua bài đánh giá dưới đây của mình . 	0
[[];[];[];[];[]]
Thông số kỹ thuật 	0
[[];[];[];[];[]]
Đánh giá chi tiết điện thoại Samsung Galaxy M51 	0
[[];[];[];[];[]]
Thiết kế 	0
[[];[];[];[];[]]
Nói về thiết kế , mình phải nói đó là một bước đột phá mạnh mẽ của Samsung . 	0
[[];[];[];[];[]]
Từ trước 

In [18]:
file_train = './train.txt'
if os.path.exists(file_train):
    os.remove(file_train)

with open(file_train, "a") as file:
    for data in train_set:
        data = add_label(data)
        data = replaceLabelType(data)
        data = removeAllEmptyLine(data)
        data = data.replace('[SUP+]', '[COM+]')
        data = data.replace('[SUP-]', '[COM-]')
        data = data.replace('[SUP]', '[COM]')
        file.write(data)
    file.close()

In [19]:
file_test = './test.txt'
if os.path.exists(file_test):
  os.remove(file_test)

with open(file_test, "a") as file:
    for data in dev_set:
        data = add_label(data)
        data = replaceLabelType(data)
        data = removeAllEmptyLine(data)
        data = data.replace('[SUP+]', '[COM+]')
        data = data.replace('[SUP-]', '[COM-]')
        data = data.replace('[SUP]', '[COM]')
        file.write(data)
    file.close()

In [20]:
file_dev = './dev.txt'
if os.path.exists(file_dev):
  os.remove(file_dev)

with open(file_dev, "a") as file:
    for data in dev_set:
        data = add_label(data)
        data = replaceLabelType(data)
        data = removeAllEmptyLine(data)
        data = data.replace('[SUP+]', '[COM+]')
        data = data.replace('[SUP-]', '[COM-]')
        data = data.replace('[SUP]', '[COM]')
        file.write(data)
    file.close()